In [ ]:
from PIL import ImageTk, Image
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
import cv2
import os
import re
from tkinter import *
from tkinter import filedialog
import tksheet
import io

In [ ]:
def adapt_array(arr):
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())

In [ ]:
def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [ ]:
# Converts np.array to TEXT when inserting
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

# Create connection to Database

In [ ]:
train_conn = sqlite3.connect('Train_Images.db', detect_types=sqlite3.PARSE_DECLTYPES)
train_connection = train_conn.cursor()

test_conn = sqlite3.connect('Test_Images.db', detect_types=sqlite3.PARSE_DECLTYPES)
test_connection = test_conn.cursor()

In [ ]:
# uniform_map is a dictionary of 58 eigenvalues of equivalent pattern, which are serialized and numbered from small to large
uniform_map = {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 6: 5, 7: 6, 8: 7, 12: 8,14: 9, 15: 10, 16: 11, 24: 12, 28: 13, 30: 14, 31: 15, 32: 16, 48: 17,
 56: 18, 60: 19, 62: 20, 63: 21, 64: 22, 96: 23, 112: 24,120: 25, 124: 26, 126: 27, 127: 28, 128: 29, 129: 30, 131: 31, 135: 32,143: 33,
 159: 34, 191: 35, 192: 36, 193: 37, 195: 38, 199: 39, 207: 40,223: 41, 224: 42, 225: 43, 227: 44, 231: 45, 239: 46, 240: 47, 241: 48,
243: 49, 247: 50, 248: 51, 249: 52, 251: 53, 252: 54, 253: 55, 254: 56,255: 57}

In [ ]:
def cal_basic_lbp(img,i,j):#Points larger than the center pixel are assigned a value of 1, and those smaller than the center pixel are assigned a value of 0. The binary sequence is returned
    sum = []
    if img[i - 1, j ] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i - 1, j+1 ] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i , j + 1] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i + 1, j+1 ] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i + 1, j ] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i + 1, j - 1] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i , j - 1] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    if img[i - 1, j - 1] > img[i, j]:
        sum.append(1)
    else:
        sum.append(0)
    return sum

In [ ]:
def bin_to_decimal(bin):#Binary to decimal
    res = 0
    bit_num = 0 #Shift left
    for i in bin[::-1]:
        res += i << bit_num   # Shifting n bits to the left is equal to multiplying by 2 to the nth power
        bit_num += 1
    return res

In [ ]:
def lbp_uniform(img):
    revolve_array = np.zeros(img.shape,np.uint8)
    width = img.shape[0]
    height = img.shape[1]
    for i in range(1,width-1):
        for j in range(1,height-1):
            sum_ = cal_basic_lbp(img,i,j) #Get binary
            num_ = calc_sum(sum_)  #Get jump times
            if num_ <= 2:
                revolve_array[i,j] = uniform_map[bin_to_decimal(sum_)] #If the number of jumps is less than or equal to 2, the decimal value corresponding to the binary sequence is the LBP value in the center of the neighborhood. There are only 58 possible values, but the maximum value can be 255, so the mapping is carried out here.
            else:
                revolve_array[i,j] = 58
    return revolve_array

In [ ]:
def calc_sum(r):  # Gets the number of jumps in the binary of the value r
    sum_ = 0
    for i in range(0,len(r)-1):
        if(r[i] != r[i+1]):
            sum_ += 1
    return sum_

# Export All data from Database

In [ ]:
train_connection.execute("SELECT * FROM train_images")
#fetchall fetchmany(any number) fetchone
train_records = train_connection.fetchall()


test_connection.execute("SELECT * FROM test_images")
#fetchall fetchmany(any number) fetchone
test_records = test_connection.fetchall()

# Convert Each Section for Processing (get size, get image)

In [ ]:
oid = 1
for train_record in train_records:
    train_row = int(re.findall(r'\b\d+\b',train_record[2])[0])
    train_column = int(re.findall(r'\b\d+\b',train_record[2])[1])
    train_image = train_record[4]
    train_image_gray = train_record[5]
    train_uniform_array = lbp_uniform(train_image_gray)
    train_connection.execute("""UPDATE train_images SET
        lbp_array':train_lbp_array
        
        WHERE oid = :oid""",
        {'lbp_array':train_lbp_array
        })
    oid += 1
oid = 1
for test_record in test_records:
    test_row = int(re.findall(r'\b\d+\b',test_record[2])[0])
    test_column = int(re.findall(r'\b\d+\b',test_record[2])[1])
    test_image = test_record[4]
    test_image_gray = test_record[5]
    test_lbp_array = lbp_uniform(test_image_gray)
    test_connection.execute("""UPDATE test_images SET
        lbp_array':test_lbp_array
        
        WHERE oid = :oid""",
        {'lbp_array':test_lbp_array
        })
    oid += 1

# Close Connection from Database

In [ ]:
train_conn.commit()
test_conn.commit()
train_conn.close()
test_conn.close()